In [ ]:
import glob
import os
import shutil

from astropy import units as u
from astropy.io import fits
from astropy.nddata import Cutout2D
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd


# Determine where the visualization will be rendered
output_notebook()

In [ ]:
## Define all Paths here: 
#biyearly_image = '/Users/cmartlin/Desktop/postflash_2022_work/2018-2019_test_01_26_2022_idle02ayq_flc.fits'
#current_image = '/Users/cmartlin/Desktop/postflash_2022_work/idle02ayq/idle02ayq_flc.fits'
#yearly_image = '/Users/cmartlin/Desktop/postflash_2022_work/2018_yearly_low_10_18_2022_idn011e6s_flc.fits'

#updated_yearly = glob.glob('/Users/cmartlin/Desktop/postflash_2022_work/*10_18*')
#science_pipeline = glob.glob('/Users/cmartlin/Desktop/postflash_2022_work/i*/i*flc.fits')

In [ ]:
## Define all Paths here: 
biyearly_image = '2022_ISR_testing_data/2018-2019_test_01_26_2022_idle02ayq_flc.fits'
current_image = '2022_ISR_testing_data/idle02ayq_flc.fits'
yearly_image = '2022_ISR_testing_data/2018_yearly_low_10_18_2022_idn011e6s_flc.fits'

updated_yearly = glob.glob('2022_ISR_testing_data/*10_18*')
science_pipeline = glob.glob('2022_ISR_testing_data/i*flc.fits')

In [ ]:
#Checking the testing data rootnames used in function:
for f in updated_yearly: 
    print(f[-18:])
    
#Checking for year length - if this needs to change, update in function below: 
for f in updated_yearly: 
    print(f[22:26])

In [ ]:
#Check the science data rootnames used in function:
for f in science_pipeline: 
    print(f[-18:])

In [ ]:
def create_databases(input_data, positions_list):
    size = (101, 101)
    
    min_pos = []
    max_pos = []
    mean_pos = []
    median_pos = []
    std_pos = []
    
    for f in input_data: 
        image_data = fits.getdata(f,1)
        means = []
        maxes = []
        mins = []
        medians = []
        stdevs = []
        for p in positions_list: 
            cutout = Cutout2D(image_data, p, size)
            cutout1 = cutout.data
            mins.append(np.min(cutout1))
            maxes.append(np.max(cutout1))
            means.append(np.mean(cutout1))
            medians.append(np.median(cutout1))
            stdevs.append(np.std(cutout1))
        std_pos.append(stdevs)
        min_pos.append(mins)
        max_pos.append(maxes)
        mean_pos.append(means)
        median_pos.append(medians)
    filenames = []
    years = []

    for f in input_data:
        filenames.append(f[-18:])
        years.append(f[44:48]) 

    positions = [220, 900, 2100, 3500, 3600]
    positions_pipeline = ['pipe_220', 'pipe_900', 'pipe_2100', 'pipe_3500', 
                          'pipe_3600']
       
    # Condition to check if there is a year in the filename:
    if (years[0].isnumeric()) == True: 
        stdev_df = pd.DataFrame(std_pos, columns=positions, dtype = float, 
                                index=filenames)
        min_df = pd.DataFrame(min_pos, columns=positions, dtype = float, 
                                    index=filenames)
        max_df = pd.DataFrame(max_pos, columns=positions, dtype = float, 
                                    index=filenames)
        mean_df = pd.DataFrame(mean_pos, columns=positions, dtype = float, 
                                    index=filenames)
        median_df = pd.DataFrame(median_pos, columns=positions, dtype = float, 
                                    index=filenames)
        years = pd.DataFrame(years, index=filenames)
        stdev_df = pd.concat([stdev_df, years],1)
        min_df = pd.concat([min_df, years],1)
        max_df = pd.concat([max_df, years],1)
        mean_df = pd.concat([mean_df, years],1)
        median_df = pd.concat([median_df, years],1)
    else:
        stdev_df = pd.DataFrame(std_pos, columns=positions_pipeline, 
                                dtype = float, index=filenames)
        min_df = pd.DataFrame(min_pos, columns=positions_pipeline, 
                              dtype = float, index=filenames)
        max_df = pd.DataFrame(max_pos, columns=positions_pipeline, 
                              dtype = float, index=filenames)
        mean_df = pd.DataFrame(mean_pos, columns=positions_pipeline, 
                               dtype = float, index=filenames)
        median_df = pd.DataFrame(median_pos, columns=positions_pipeline, 
                                 dtype = float, index=filenames)
    
    #return stdev_df, min_df, max_df, mean_df, median_df
    return stdev_df

In [ ]:
positions_list = [(220, 160), (900,700), (2100,1100), (3500,300), (3600,1700)]

stdev_yearly = create_databases(updated_yearly, positions_list)
stdev_pipeline = create_databases(science_pipeline, positions_list)

combine_by_file = pd.concat([stdev_yearly,stdev_pipeline],1)
stdev_yearly

In [ ]:
combine_by_file['220_norm'] = (combine_by_file[220] - combine_by_file['pipe_220']) / combine_by_file['pipe_220']
combine_by_file['900_norm'] = (combine_by_file[900] - combine_by_file['pipe_900']) / combine_by_file['pipe_900']
combine_by_file['2100_norm'] = (combine_by_file[2100] - combine_by_file['pipe_2100']) / combine_by_file['pipe_2100']
combine_by_file['3500_norm'] = (combine_by_file[3500] - combine_by_file['pipe_3500']) / combine_by_file['pipe_3500']
combine_by_file['3600_norm'] = (combine_by_file[3600] - combine_by_file['pipe_3600']) / combine_by_file['pipe_3600']

stdev_sorted_by_year = combine_by_file.sort_values(by=[0])
stdev_sorted_by_year

In [ ]:
x = list([2012,2013,2014,2015,2016,2017,2018,2019,2020,2021])
#x2 = list([5,1,4,6,3,2])
x3 = list([1, 2, 3, 4, 5, 6])
y = list(sorted_by_year['220_norm'].values)
y2 = list(sorted_by_year['900_norm'].values)
y3 = list(sorted_by_year['2100_norm'].values)
y4 = list(sorted_by_year['3500_norm'].values)
y5 = list(sorted_by_year['3600_norm'].values)

p = figure(title = 'Checking Stdev')
p.line(x,y)
p.line(x,y2)
p.line(x,y3)
p.line(x,y4)
p.line(x,y5)
show(p)

In [ ]:
image_biyearly = fits.getdata(biyearly_image,1)
image_curr = fits.getdata(current_image,1)
image_yearly = fits.getdata(yearly_image,1)

figure(figsize=(10, 8), dpi=80)

plt.hist(image_biyearly.flatten(),alpha=0.5, range=(-20,20), bins=100)
plt.hist(image_curr.flatten(),alpha=0.5, range=(-20,20), bins=100)
plt.hist(image_yearly.flatten(),alpha=0.5, range=(-20,20), bins=100)

In [ ]:
position = (220,160)
size = (101, 101)

cutout_biyearly = Cutout2D(image_biyearly, position , size)
cutout1_biyearly = cutout_biyearly.data

cutout_yearly = Cutout2D(image_yearly, position , size)
cutout1_yearly = cutout_yearly.data

cutout_curr = Cutout2D(image_curr, position , size)
cutout1_curr = cutout_curr.data

figure(figsize=(10, 8), dpi=80)

plt.hist(image_biyearly.flatten(),alpha=0.5, range=(-20,20), bins=200)
plt.hist(image_curr.flatten(),alpha=0.5, range=(-20,20), bins=200)
plt.hist(image_yearly.flatten(),alpha=0.5, range=(-20,20), bins=200)